In [24]:
# Install requirements, uncomment to run
# !pip install geopandas pillow label-studio-converter

# Uploading Savmap dataset to label studio

In [2]:
import geopandas as gdp
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path
import numpy as np
import json
import pandas as pd
from torchvision.ops import nms
import torch
from label_studio_sdk import Client
from dotenv import load_dotenv
from label_studio_tools.core.utils.io import get_local_path
from urllib.parse import unquote
import traceback

In [3]:
#  load data
root = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data"
polygons = gdp.read_file(os.path.join(root,'savmap_annotations_2014.geojson'))

path_to_images = os.path.join(root,'images')
path_to_labels = os.path.join(root,'labels')

In [ ]:
# # polygon points
# xx,yy = polygons['geometry'].iloc[0].exterior.coords.xy
# np.array(list(zip(xx,yy))) * 100 / np.array([4000,3000]).reshape((1,2))

In [7]:
# get bounding boxs
for cat in ['x_min', 'y_min', 'x_max', 'y_max','width','height','x','y']:
    polygons[cat] = None
for i in tqdm(range(len(polygons))):
    x_min, y_min, x_max, y_max = polygons['geometry'].iloc[i].bounds
    image_path = os.path.join(path_to_images,f"{polygons['IMAGEUUID'].iloc[i]}.JPG")
    # try:
    width, height = Image.open(image_path).size
    # except:
    #     continue
    polygons['x_min'].iat[i] = max(int(x_min),0)
    polygons['x_max'].iat[i] = max(int(x_max),0)
    polygons['y_min'].iat[i] = max(int(y_min),0)
    polygons['y_max'].iat[i] = max(int(y_max),0)
    polygons['x'].iat[i] = 0.5*(x_max+x_min)
    polygons['y'].iat[i] = 0.5*(y_max+y_min)
    polygons['width'].iat[i] = width
    polygons['height'].iat[i] = height

polygons['bbox_w'] = (polygons['x_max'] - polygons['x_min'])*1.0
polygons['bbox_h'] = (polygons['y_max'] - polygons['y_min'])*1.0
polygons['class'] = 0

100%|██████████| 7474/7474 [00:03<00:00, 2149.95it/s]


In [8]:
# non-max suppresion
# https://pytorch.org/vision/stable/generated/torchvision.ops.nms.html

def nms_to_bbox(df_annotations:pd.DataFrame,iou_threshold:float=0.5):
    dfs = list()

    for IMAGEUUID, df in tqdm(df_annotations.groupby('IMAGEUUID')):

        df.reset_index(inplace=True)

        bbox = df[['x_min','y_min','x_max','y_max']].to_numpy().astype(float)
        scores = np.ones((bbox.shape[0],)).astype(float)

        bbox_indices= nms(boxes=torch.from_numpy(bbox),
                            scores=torch.from_numpy(scores),
                            iou_threshold=iou_threshold).numpy()
        
        # print(bbox[bbox_indices],'\n',df.iloc[bbox_indices,:])

        dfs.append(df.iloc[bbox_indices,:].copy())

    return pd.concat(dfs,axis=0).reset_index()

In [9]:
# nms filtering of bbox
df_filtered = nms_to_bbox(polygons,iou_threshold=0.4)
df_filtered.rename(columns={'class':'labels'},inplace=True)

  0%|          | 0/654 [00:00<?, ?it/s]

100%|██████████| 654/654 [00:00<00:00, 1153.51it/s]


In [10]:
df_filtered.drop(columns=['level_0','index'],inplace=True)

In [11]:
# filtered df vs unfiltered df
len(df_filtered), len(polygons)

(3313, 7474)

In [12]:
df_filtered.head(1)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,labels
0,003a34ee6b7841e6851b8fe511ebe102,d57402df14a84b58a411bade4f9d14f1,"POLYGON ((1514 962, 1524 970, 1536 968, 1536 9...",1503,962,1537,996,4000,3000,1520.0,979.0,34.0,34.0,0


In [ ]:
# save filtered annotations as csv
savepath = os.path.join(root,'savmap_annotations_2014_filtered.csv')
df_filtered['images'] = df_filtered['IMAGEUUID'].apply(lambda x: f"{x}.JPG")
df_filtered.to_csv(savepath,index=False)

In [5]:
savepath = os.path.join(root,'savmap_annotations_2014_filtered.csv')
df_filtered = pd.read_csv(savepath)
df_filtered.head(2)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,labels,images
0,003a34ee6b7841e6851b8fe511ebe102,d57402df14a84b58a411bade4f9d14f1,"POLYGON ((1514 962, 1524 970, 1536 968, 1536 9...",1503,962,1537,996,4000,3000,1520.0,979.0,34.0,34.0,0,003a34ee6b7841e6851b8fe511ebe102.JPG
1,003a34ee6b7841e6851b8fe511ebe102,ece6f29e46544b4eaeb4c057c0ecfe06,"POLYGON ((1688 858, 1695 865, 1694 870, 1692 8...",1679,858,1695,875,4000,3000,1687.0,866.5,16.0,17.0,0,003a34ee6b7841e6851b8fe511ebe102.JPG


In [15]:
# Tiling images
!python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images" 2000 2000 100 -dest "D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images_splits" -csv "D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\savmap_annotations_2014_filtered.csv" -min 0.5 -all False


Creating the buffer: 100%|██████████| 654/654 [01:11<00:00,  9.09it/s]

Exporting patches: 100%|██████████| 654/654 [06:06<00:00,  1.78it/s]


In [13]:
# load groundtruth for split images
path_to_images_splits = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images_splits"
df_splits = pd.read_csv(os.path.join(path_to_images_splits,"..\gt_splits.csv"))

try:
    df_splits["IMAGEUUID"] = df_splits['images'].apply(lambda x: str(x).split(".")[0])
    df_splits['x'] = df_splits["x_min"] #+df_splits["x_max"])
    df_splits['y'] = df_splits["y_min"] #+df_splits["y_max"])
    df_splits["width"] = 0.
    df_splits["height"] = 0.
    df_splits['bbox_w'] = df_splits['x_max'] - df_splits['x_min']
    df_splits['bbox_h'] = df_splits['y_max'] - df_splits['y_min']
    
except Exception as e:
    traceback.print_exc()
    df_splits['x'] = df_splits['x'] - 0.5*df_splits['bbox_w']
    df_splits['y'] = df_splits['y'] - 0.5*df_splits['bbox_h']


for name,df in tqdm(df_splits.groupby("images")):
    image_path = os.path.join(path_to_images_splits,name)
    width, height = Image.open(image_path).size
    mask = (df_splits["images"] == name)
    df_splits.loc[mask,'width']  = width
    df_splits.loc[mask,'height'] = height

<>:3: SyntaxWarning: invalid escape sequence '\g'
<>:3: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Machine Learning\AppData\Local\Temp\ipykernel_16696\2608163779.py:3: SyntaxWarning: invalid escape sequence '\g'
  df_splits = pd.read_csv(os.path.join(path_to_images_splits,"..\gt_splits.csv"))
Traceback (most recent call last):
  File "c:\Users\Machine Learning\anaconda3\envs\label-backend\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'x_min'

The above exception was the direct cause of t

In [14]:
df_splits.head(3)

,images,labels,base_images,IMAGEUUID,TAGUUID,geometry,width,height,bbox_w,bbox_h,x,y
0,003a34ee6b7841e6851b8fe511ebe102_0.JPG,0,003a34ee6b7841e6851b8fe511ebe102.JPG,003a34ee6b7841e6851b8fe511ebe102_0,d57402df14a84b58a411bade4f9d14f1,"POLYGON ((1514 962, 1524 970, 1536 968, 1536 9...",2000,2000,34.0,34.0,1503.0,962.0
1,003a34ee6b7841e6851b8fe511ebe102_0.JPG,0,003a34ee6b7841e6851b8fe511ebe102.JPG,003a34ee6b7841e6851b8fe511ebe102_0,ece6f29e46544b4eaeb4c057c0ecfe06,"POLYGON ((1688 858, 1695 865, 1694 870, 1692 8...",2000,2000,16.0,17.0,1679.0,858.0
2,0078d29a8d0b489caa3425969c7477ac_0.JPG,0,0078d29a8d0b489caa3425969c7477ac.JPG,0078d29a8d0b489caa3425969c7477ac_0,bf058081d2cd4e708a0f590a16a4e16e,"POLYGON ((620 1674, 624 1684, 638 1689, 647 16...",2000,2000,57.0,36.0,620.0,1653.0


In [ ]:
# Select labels
# data = polygons.loc[:,['x_min','x_max','y_min','y_max','IMAGEUUID','width','height','class']].copy()
# data.rename(columns={'IMAGEUUID':'filename',
#                      'x_max':'xmax',
#                      'x_min':'xmin',
#                      'y_min':'ymin',
#                      'y_max':'ymax'},inplace=True)
# data['filename'] = data['filename'].apply(lambda x: f"{x}.JPG")
# data = data.dropna()
# data = data[['filename','class','width', 'height','xmin','ymin','xmax','ymax']]
# data.head(1)


In [ ]:
# # Convert CSV to COCO
# save_json_path = ... #'cocoformat.json'

# images = []
# categories = []
# annotations = []

# category = {}
# category["supercategory"] = 'None'
# category["id"] = 0
# category["category_name"] = 'wildlife'
# categories.append(category)

# data['fileid'] = data['filename'].astype('category').cat.codes
# data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
# # data['categoryid'] = data['categoryid']+1
# data['annid'] = data.index

# def image(row):
#     image = {}
#     image["height"] = row.height
#     image["width"] = row.width
#     image["id"] = row.fileid
#     image["file_name"] = row.filename
#     return image

# # def category(row):
# #     category = {}
# #     category["supercategory"] = 'None'
# #     category["id"] = row.categoryid
# #     category["category_name"] = row[2]
# #     return category

# def annotation(row):
#     annotation = {}
#     area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
#     annotation["segmentation"] = []
#     annotation["iscrowd"] = 0
#     annotation["area"] = area
#     annotation["image_id"] = row.fileid
#     annotation["score"] = 1.0

#     annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

#     annotation["category_id"] = row.categoryid
#     annotation["id"] = row.annid
#     return annotation

# for row in data.itertuples():
#     annotations.append(annotation(row))

# imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
# for row in imagedf.itertuples():
#     images.append(image(row))

# # catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
# # for row in catdf.itertuples():
# #     categories.append(category(row))

# data_coco = {}
# data_coco["images"] = images
# data_coco["categories"] = categories
# data_coco["annotations"] = annotations
# # json.dump(data_coco, open(save_json_path, "w"), indent=4)

In [ ]:
# groundtruth = {annot['file_name']:[] for annot in data_coco['images']}
# for annot,image_data in zip(data_coco['annotations'],data_coco['images']):
#     annot.update(image_data)
#     # pprint.pp(annot)
#     annot['category_name'] = category['category_name']
#     groundtruth[annot['file_name']].append(annot)

In [6]:
dotenv_path = r"..\.env"
if dotenv_path is not None:
    load_dotenv(dotenv_path=dotenv_path)

In [7]:
# Connect to the Label Studio API and check the connection
LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL')
API_KEY = os.getenv("LABEL_STUDIO_API_KEY")
labelstudio_client = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

In [ ]:
# url = "/data/local-files/?d=Users%5Cfadel%5COneDrive%5CBureau%5CWILD-AI%5Cdatalabeling%5Cdata%5Ctrain_wildai%5Cimages%5C003a34ee6b7841e6851b8fe511ebe102.JPG"

In [ ]:
# unquote(url)

In [82]:
# Uploading Polygons
def format_prediction_polygon(points:list,width:int,height:int,label:list=['wildlife',]):
    
    template = {
            "original_width": width,
            "original_height": height,
            "image_rotation": 0,
            "value": {
                "points": points,
                "closed":True,
                "polygonlabels": label
            }
    }

    return template

def format_predictions_polygon(polygons:list,widths:list,heights:list,label:list=['wildlife',]):

    results = list()
    for polygon,w,h in zip(polygons,widths,heights):
        xx,yy=polygon.exterior.coords.xy
        points = list(zip(xx,yy))

        # convert points to percent
        points = np.array(points) * 100 / np.array([w,h]).reshape((1,2))
        points = points.tolist()      

        # append result
        results.append(format_prediction_polygon(points,width=w,height=h,label=label))

    return results


# uploading polygons
 # Select project
# project = labelstudio_client.get_project(id=project_id)
# # Upload predictions for each task
# tasks = project.get_tasks()
# if top_n > 0:
#     tasks = sorted(tasks,key=lambda x:x['id'])[:top_n]

for task in tqdm(tasks[:10],desc="Uploading predictions"):
    task_id = task['id']
    img_url = unquote(task['data']['image'])
    img_path = get_local_path(img_url)
    img_name = Path(img_path).stem
    mask = polygons['IMAGEUUID']==img_name
    polys = polygons.loc[mask,['IMAGEUUID','geometry','width','height']]
    formatted_pred = format_predictions_polygon(polygons=polys['geometry'],widths=polys['width'],heights=polys['height'],label=['wildlife',])
    # conf_scores = [pred['score'] for pred in prediction]
    # max_score = 0.0
    project.create_prediction(task_id=task_id,
                               result=formatted_pred,model_version='gt')

In [28]:
import traceback

# uploading gt bbox
def format_prediction_bbox(pred:dict,img_height:int,img_width:int,
                           from_name:str='label',
                           to_name:str='image',
                           label_type:str='rectanglelabels') -> dict:
        """Converts prediction Label studio format

        Args:
            pred (dict): prediction in coco format
            img_height (int): image height
            img_width (int): image width

        Returns:
            dict: Label studio formated prediction
        """
        # formatting the prediction to work with Label studio
        x, y, width, height = pred['bbox']
        label = pred['category_name']
        # score = pred['score']
        # if not isinstance(score,float):
        #     score = 0.0
        template = {
                    "from_name": from_name,
                    "to_name": to_name,
                    "type": label_type,
                    "original_width":img_width,
                    "original_height":img_height,
                    "image_rotation":0,
                    'value': {
                        label_type: [label,],
                        'x': x / img_width * 100.,
                        'y': y / img_height * 100.,
                        'width': width / img_width * 100.,
                        'height': height / img_height * 100.,
                        'rotation':0.
                    },
                    # 'score': score
        }
        return template

def format_predictions_bbox(xs:list,ys:list,ws:list,hs:list,img_width:int,img_height:int,label:str='wildlife'):
      
    results = list()
    for x,y,w,h in zip(xs,ys,ws,hs):
        
        annot = {'bbox':[x,y,w,h],
                 'category_name':label,
                 'score':None
                 }

        # append result
        results.append(format_prediction_bbox(annot,img_width=img_width,img_height=img_height))

    return results

def upload_predictions_bbox(df_annotations:pd.DataFrame,project_id:int):

    # get tasks
    project = labelstudio_client.get_project(id=project_id)
    tasks = project.get_tasks()

    # upload
    failed_uploads = set()
    for task in tqdm(tasks[:],desc="Uploading predictions"):
        task_id = task['id']
        img_url = unquote(task['data']['image'])
        img_path = get_local_path(img_url)
        img_name = Path(img_path).stem
        try:
            mask = df_annotations['IMAGEUUID']==img_name
            df_bbox = df_annotations.loc[mask,['x','y','bbox_w','bbox_h','width','height']]
            img_width = df_bbox['width'].iat[0]
            img_height = df_bbox['height'].iat[0]
            formatted_pred = format_predictions_bbox(xs=df_bbox['x'],
                                                    ys=df_bbox['y'],
                                                    ws=df_bbox['bbox_w'],
                                                    hs=df_bbox['bbox_h'],
                                                    img_width=int(img_width),
                                                    img_height=int(img_height),
                                                    label='wildlife'
                                                    )
            # project.create_annotation(task_id=task_id,
            #                         result=formatted_pred,
            #                         # model_version='gt'
            #                         )
            project.create_prediction(task_id=task_id,
                                    result=formatted_pred,
                                    model_version='gt')
        except Exception as e:
            # skipping empty df
            if len(df_bbox)<1:
                failed_uploads.add(task_id)
                continue
            else:
                print("Failed for: ",img_name,df_bbox,end="\n")
                traceback.print_exc()
                break
    return failed_uploads

def delete_tasks(project_id:int,failed_uploads:set[int]):

    # get project
    project = labelstudio_client.get_project(id=project_id)

    # delete tasks
    for task_id in tqdm(failed_uploads,desc="deleting tasks"):
        project.delete_task(task_id=task_id)

In [18]:
# savmap original
project_id = 71
failed_uploads = upload_predictions_bbox(df_annotations=df_filtered,project_id=project_id)

Uploading predictions: 100%|██████████| 654/654 [00:23<00:00, 27.51it/s]


In [19]:
len(failed_uploads)

0

In [29]:
# savmap splits
project_id = 70
failed_uploads = upload_predictions_bbox(df_annotations=df_splits,project_id=project_id)

Uploading predictions: 100%|██████████| 1162/1162 [00:42<00:00, 27.36it/s]


In [30]:
len(failed_uploads)

0

In [27]:
# delete empty images
delete_tasks(project_id=project_id,failed_uploads=failed_uploads)

deleting tasks: 100%|██████████| 3924/3924 [02:33<00:00, 25.60it/s]


In [79]:
# format_predictions(polygons=polys['geometry'],widths=polys['width'],heights=polys['height'],label=['wildlife',])

# Formatting savmap dataset
Formatting the savmap dataset so it can be imported in label studio through a json file

In [1]:
import geopandas as gdp
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path

In [2]:
#  load data
root = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data"
polygons = gdp.read_file(os.path.join(root,'savmap_annotations_2014.geojson'))

# create directories
path_to_images = os.path.join(root,'images')
path_to_labels = os.path.join(root,'labels')
Path(path_to_images).mkdir(exist_ok=True,parents=True)
Path(path_to_labels).mkdir(exist_ok=True,parents=True)

In [3]:
# Move files
# ! mv ../data/savmap_dataset_v2/*.JPG ../data/savmap_dataset_v2/images

In [3]:
# get bounding boxs
for cat in ['x_min', 'y_min', 'x_max', 'y_max','width','height','x','y']:
    polygons[cat] = None
for i in range(len(polygons)):
    x_min, y_min, x_max, y_max = polygons['geometry'].iloc[i].bounds
    image_path = os.path.join(path_to_images,f"{polygons['IMAGEUUID'].iloc[i]}.JPG")
    width, height = Image.open(image_path).size
    polygons['x_min'].iat[i] = max(0,int(x_min))
    polygons['x_max'].iat[i] = max(0,int(x_max))
    polygons['y_min'].iat[i] = max(0,int(y_min))
    polygons['y_max'].iat[i] = max(0,int(y_max))
    polygons['x'].iat[i] = 0.5*(x_max+x_min)
    polygons['y'].iat[i] = 0.5*(y_max+y_min)
    polygons['width'].iat[i] = width
    polygons['height'].iat[i] = height

# creat bbox width and height
polygons['bbox_w'] = polygons['x_max'] - polygons['x_min']
polygons['bbox_h'] = polygons['y_max'] - polygons['y_min']
polygons['class'] = 0

In [4]:
polygons.head(1)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,class
0,f77f4af5a1344b9086b307d2b4ba61ff,a9b3a2325dbe4a208bc3ae37eeb8e1e1,"POLYGON ((1197 568, 1186 568, 1179 582, 1190 5...",1179,568,1230,597,4000,3000,1204.5,582.5,51,29,0


In [94]:
# convert to yolo format
def save_df_as_yolo(df_annotation:gdp.GeoDataFrame,dest_path_labels:str):
    
    cols = ['class','x','y','bbox_w','bbox_h']
    for col in cols:
        assert df_annotation[col].isna().sum()<1,'there are NaN values. Check out.'
        # df_annotation[col] = df_annotation[col].apply(int)

    # normalize values
    df_annotation.loc[:,'x'] = df_annotation['x']/df_annotation['width']
    df_annotation.loc[:,'y'] = df_annotation['y']/df_annotation['height']
    df_annotation.loc[:,'bbox_w'] = df_annotation['bbox_w']/df_annotation['width']
    df_annotation.loc[:,'bbox_h'] = df_annotation['bbox_h']/df_annotation['height']
    
    for image_name,df in tqdm(df_annotation.groupby('IMAGEUUID'),desc='Saving yolo labels'):
        txt_file = f'{image_name}.txt'
        df[cols].to_csv(os.path.join(dest_path_labels,txt_file),sep=' ',index=False,header=False)

In [95]:
save_df_as_yolo(df_annotation=polygons,dest_path_labels=path_to_labels)

Saving yolo labels:   0%|          | 0/654 [00:00<?, ?it/s]

Saving yolo labels: 100%|██████████| 654/654 [00:01<00:00, 463.09it/s]


In [ ]:
# tutorial here: https://github.com/HumanSignal/label-studio-converter/tree/master
# if it does not work, use a terminal
# !label-studio-converter import yolo -i "C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai" --image-ext ".JPG" --out-type "predictions" -o "C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\ls_tasks.json"

# Formatting General Dataset from Delplanque 2021

In [9]:
import pandas as pd
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm


## Tiling

In [ ]:
# Tiling test
# !python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\general_dataset\test" 640 640 64  -dest "D:\PhD\Data per camp\Extra training data\general_dataset\tiled_data\test_tiled\images" -pattern "**/*.JPG" -csv "D:\PhD\Data per camp\Extra training data\general_dataset\groundtruth\csv\test_big_size_A_B_E_K_WH_WB.csv"

In [ ]:
# Tiling val
# !python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\general_dataset\val" 640 640 64  -dest "D:\PhD\Data per camp\Extra training data\general_dataset\val_tiled" -pattern "**/*.JPG" -csv "D:\PhD\Data per camp\Extra training data\general_dataset\groundtruth\csv\val_big_size_A_B_E_K_WH_WB.csv"

In [ ]:
# Tiling Train
# !python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\general_dataset\train" 640 640 64  -dest "D:\PhD\Data per camp\Extra training data\general_dataset\train_tiled" -pattern "**/*.JPG" -csv "D:\PhD\Data per camp\Extra training data\general_dataset\groundtruth\csv\train_big_size_A_B_E_K_WH_WB.csv"

## Converting to YOLO format

In [56]:
# convert to yolo format
def save_df_as_yolo(df_annotation:pd.DataFrame,dest_path_labels:str,is_detector:bool=False):
    
    cols = ['class','x','y','bbox_w','bbox_h']
    for col in cols:
        assert df_annotation[col].isna().sum()<1,'there are NaN values. Check out.'
        # df_annotation[col] = df_annotation[col].apply(int)
    
    for col in ['x','y','bbox_w','bbox_h']:
        df_annotation[col] = df_annotation[col].astype(float)

    # normalize values
    df_annotation.loc[:,'x'] = df_annotation['x']/df_annotation['width']
    df_annotation.loc[:,'y'] = df_annotation['y']/df_annotation['height']
    df_annotation.loc[:,'bbox_w'] = df_annotation['bbox_w']/df_annotation['width']
    df_annotation.loc[:,'bbox_h'] = df_annotation['bbox_h']/df_annotation['height']

    if is_detector:
        df_annotation['class'] = 0
    
    for image_name,df in tqdm(df_annotation.groupby('images'),desc='Saving yolo labels'):
        txt_file = f'{Path(image_name).stem}.txt'
        df[cols].to_csv(os.path.join(dest_path_labels,txt_file),sep=' ',index=False,header=False)

In [67]:
split='train' # 'train' 'test', 'val
root = Path(rf"D:\PhD\Data per camp\Extra training data\general_dataset\tiled_data\{split}_tiled")
path_to_csv = root/"gt.csv"
path_images = root/'images'
path_to_labels = root/'labels'
detection_mode=False # save label for wildlife detection only

In [68]:
df_annotations = pd.read_csv(path_to_csv)
# df_annotations.head(2)

# update df_annotations
df_annotations['width'] = 0.0
df_annotations['height'] = 0.0


for name in set(df_annotations.images):
    width, height = Image.open(root/f"images/{name}").size
    df_annotations.loc[df_annotations.images==name,'width'] = float(width)
    df_annotations.loc[df_annotations.images==name,'height'] = float(height)

df_annotations['x'] = 0.5*(df_annotations['x_min'] + df_annotations['x_max'])
df_annotations['y'] = 0.5*(df_annotations['y_min'] + df_annotations['y_max'])
df_annotations['bbox_h'] = df_annotations['y_max'] - df_annotations['y_min']
df_annotations['bbox_w'] = df_annotations['x_max'] - df_annotations['x_min']

df_annotations.rename(columns={'labels':'class'},inplace=True)

In [69]:
# df_annotations.head(2)
# df_annotations['class'].plot(kind='hist')

In [ ]:
save_df_as_yolo(df_annotations,path_to_labels,is_detector=detection_mode)

# WAID 

## Trasforming labels for detector training

In [17]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [ ]:
path = r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\data\WAID\labels"
path = Path(path)

for p in tqdm(path.glob("*/**/*.txt")):
    df = pd.read_csv(p,sep=" ",header=None)
    df.columns = ["class",'x','y','w','h']
    df['class'] = 0
    # un-comment to run
    # df.to_csv(p, sep=" ", header=False, index=False)

# Hard sample mining

In [33]:
from dotenv import load_dotenv
load_dotenv("../.env")

from datalabeling.annotator import Detector, Annotator
from datalabeling.dataset.sampling import (get_preds_targets, 
                                           select_hard_samples,
                                           compute_detector_performance)
import pandas as pd
import yaml
import os

In [53]:
# Define detector
# to speed up inference on intel, make
# model = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
#                 confidence_threshold=0.1,
#                 overlap_ratio=0.1,
#                 tilesize=1280,
#                 device='CPU',
#                 use_sliding_window=True,
#                 is_yolo_obb=True)

# provide correct alias, "pt", "onnx"
alias = "last"
name = "obb-detector" # detector, "obb-detector"
handler = Annotator(mlflow_model_alias=alias,
                    mlflow_model_name=name,
                    confidence_threshold=0.1,
                    is_yolo_obb=name.strip() == "obb-detector",
                    dotenv_path="../.env")

11/18/2024 11:44:08 - INFO - datalabeling.annotator.models -   Computing device: cuda:0


In [ ]:
# Get detector for mlflow registered model
model = handler.model.unwrap_python_model().detection_model

In [55]:
# load groundtruth
with open(r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\data\dataset_labeler.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)

train_images_path = [os.path.join(yolo_config['path'],yolo_config['train'][i]) for i in range(len(yolo_config['train']))]
train_labels_path = [p.replace('images','labels') for p in train_images_path]

In [56]:
train_images_path,train_labels_path

(['D:\\PhD\\Data per camp\\DetectionDataset\\delplanque_tiled_data\\train_tiled\\images',
  'D:\\PhD\\Data per camp\\DetectionDataset\\delplanque_tiled_data\\val_tiled\\images',
  'D:\\PhD\\Data per camp\\DetectionDataset\\Rep 1\\train\\images',
  'D:\\PhD\\Data per camp\\DetectionDataset\\Rep 2\\train\\images',
  'D:\\PhD\\Data per camp\\DetectionDataset\\WAID\\images\\valid',
  'D:\\PhD\\Data per camp\\DetectionDataset\\WAID\\images\\test',
  'D:\\PhD\\Data per camp\\DetectionDataset\\Rep 2\\val\\images',
  'D:\\PhD\\Data per camp\\DetectionDataset\\Rep 1\\val\\images',
  'D:\\PhD\\Data per camp\\DetectionDataset\\bushriver\\images'],
 ['D:\\PhD\\Data per camp\\DetectionDataset\\delplanque_tiled_data\\train_tiled\\labels',
  'D:\\PhD\\Data per camp\\DetectionDataset\\delplanque_tiled_data\\val_tiled\\labels',
  'D:\\PhD\\Data per camp\\DetectionDataset\\Rep 1\\train\\labels',
  'D:\\PhD\\Data per camp\\DetectionDataset\\Rep 2\\train\\labels',
  'D:\\PhD\\Data per camp\\DetectionDatas

In [ ]:
df_results, df_labels, col_names = get_preds_targets(images_dirs=train_images_path,
                                                  pred_results_dir="D:\PhD\Data per camp\DetectionDataset\hard_samples",
                                                  detector=model,
                                                  load_results=False)

0it [00:00, ?it/s]


0: 1280x1280 22.1ms
Speed: 6.0ms preprocess, 22.1ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)


1it [00:00,  1.12it/s]


0: 1280x1280 24.1ms
Speed: 5.0ms preprocess, 24.1ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 23.1ms
Speed: 7.0ms preprocess, 23.1ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 23.1ms
Speed: 6.0ms preprocess, 23.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)


4it [00:01,  4.83it/s]

...

735it [00:30, 24.04it/s]


0: 1280x1280 (no detections), 22.1ms
Speed: 7.0ms preprocess, 22.1ms inference, 0.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 24.1ms
Speed: 6.0ms preprocess, 24.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 22.1ms
Speed: 7.0ms preprocess, 22.1ms inference, 4.0ms postprocess per image at shape (1, 3, 1280, 1280)


738it [00:30, 24.22it/s]


0: 1280x1280 24.1ms
Speed: 6.0ms preprocess, 24.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 23.1ms
Speed: 6.0ms preprocess, 23.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)


740it [00:31, 23.86it/s]
pred results as df: 100%|██████████| 740/740 [00:00<00:00, 3247.28it/s]


In [58]:
df_labels.head(2)

,category_id,x1,y1,x2,y2,x3,y3,x4,y4,image_path,width,height
0,0,554.0,280.0,607.0,280.0,607.0,308.0,554.0,308.0,D:\PhD\Data per camp\DetectionDataset\delplanq...,640,640
1,0,358.0,208.0,392.0,208.0,392.0,265.0,358.0,265.0,D:\PhD\Data per camp\DetectionDataset\delplanq...,640,640


In [59]:
df_results.head(2)

,score,category_id,category_name,area,image_path,x_min,y_min,bbox_w,bbox_h,x_max,y_max
0,0.828369,0.0,wildlife,None,D:\PhD\Data per camp\DetectionDataset\delplanq...,558.072083,281.108551,46.536133,25.569519,604.608215,306.678070
1,0.828369,0.0,wildlife,None,D:\PhD\Data per camp\DetectionDataset\delplanq...,390.541565,428.976929,79.589905,65.960266,470.131470,494.937195


In [60]:
col_names

['category_id', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']

In [61]:
df_results_per_img = compute_detector_performance(df_results,df_labels,col_names)
df_results_per_img.head(2)

 58%|█████▊    | 6281/10851 [10:47<17:00,  4.48it/s]c:\Users\Machine Learning\anaconda3\envs\label-backend\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028
 70%|██████▉   | 7549/10851 [15:25<12:03,  4.57it/s]c:\Users\Machine Learning\anaconda3\envs\label-backend\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_thre

,map50,map75,max_scores,image_paths
0,0.663366,0.663366,0.828369,D:\PhD\Data per camp\DetectionDataset\delplanq...
1,1.000000,1.000000,0.690774,D:\PhD\Data per camp\DetectionDataset\delplanq...


In [63]:
# # load results
# df_results_per_img = pd.read_csv(r"..\data\negative-samples\df_results_per_img.csv")

In [73]:
#uncomment to save
df_results_per_img.to_csv(r"D:\PhD\Data per camp\DetectionDataset\hard_samples\df_results_per_img.csv",index=False)

In [72]:
# save hard samples
df_hard_samples = select_hard_samples(df_results_per_img=df_results_per_img,
                                            map_thrs=0.3,
                                            score_thrs=0.7,
                                            save_path_samples=r"D:\PhD\Data per camp\DetectionDataset\hard_samples\hard_samples.txt",
                                            root='D:\\',
                                            save_data_yaml=r"D:\PhD\Data per camp\DetectionDataset\hard_samples\hard_samples.yaml"
                                            )

In [68]:
df_hard_samples.describe()

,map50,map75,max_scores
count,766.000000,766.000000,766.000000
mean,0.098118,0.040718,0.795491
std,0.112832,0.078647,0.047138
min,0.000000,0.000000,0.700399
25%,0.000000,0.000000,0.759734
50%,0.000000,0.000000,0.798060
75%,0.214109,0.039604,0.834118
max,0.299505,0.288543,0.890802


In [69]:
df_hard_samples

,map50,map75,max_scores,image_paths
2,0.000000,0.000000,0.753583,D:\PhD\Data per camp\DetectionDataset\delplanq...
5,0.000000,0.000000,0.825133,D:\PhD\Data per camp\DetectionDataset\delplanq...
12,0.252475,0.252475,0.735655,D:\PhD\Data per camp\DetectionDataset\delplanq...
29,0.128713,0.000000,0.740626,D:\PhD\Data per camp\DetectionDataset\delplanq...
47,0.257426,0.257426,0.707460,D:\PhD\Data per camp\DetectionDataset\delplanq...
...,...,...,...,...
10724,0.000000,0.000000,0.705357,D:\PhD\Data per camp\DetectionDataset\bushrive...
10734,0.000000,0.000000,0.734613,D:\PhD\Data per camp\DetectionDataset\bushrive...
10774,0.000000,0.000000,0.771991,D:\PhD\Data per camp\DetectionDataset\bushrive...
10815,0.000000,0.000000,0.746329,D:\PhD\Data per camp\DetectionDataset\bushrive...


# Converting datasets to yolo<>OBB

In [63]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

def convert_yolo_to_obb(yolo_dataset_dir:str,output_dir:str)->None:

    cols = ['id','x1','y1','x2','y2','x3','y3','x4','y4']
    names = ['id','x','y','w','h']

    # Iterate through labels
    for label_path in tqdm(Path(yolo_dataset_dir).glob("*.txt"),desc='yolo->obb'):
        df = pd.read_csv(label_path,sep=' ',names=names)

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        for col in names[1:]:
            df[col] = df[col].astype(float)
        df['id'] = df['id'].astype(int)

        df['w'] = 0.5*df['w']
        df['h'] = 0.5*df['h']
        # top left
        df['x1'] = df['x'] - df['w']
        df['y1'] = df['y'] - df['h']
        # top right
        df['x2'] = df['x'] + df['w']
        df['y2'] = df['y'] - df['h']
        # bottom right
        df['x3'] = df['x'] + df['w']
        df['y3'] = df['y'] + df['h']
        # bottom left
        df['x4'] = df['x'] - df['w']
        df['y4'] = df['y'] + df['h']

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        # save file
        df[cols].to_csv(Path(output_dir)/label_path.name,
                        sep=' ',index=False,header=False)

def convert_obb_to_yolo(obb_dataset_dir:str,output_dir:str)->None:

    names = ['id','x1','y1','x2','y2','x3','y3','x4','y4']
    cols = ['id','x','y','w','h']

    # Iterate through labels
    for label_path in tqdm(Path(obb_dataset_dir).glob("*.txt"),desc='obb->yolo'):
        df = pd.read_csv(label_path,sep=' ',names=names)

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        # center
        df['x'] = (df['x1'] + df['x2'])/2.
        df['y'] = (df['y1'] + df['y4'])/2.
        # width
        df['w'] = df['x2'] - df['x1']
        # height
        df['h'] = df['y4'] - df['y1']

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        # save file
        df[cols].to_csv(Path(output_dir)/label_path.name,sep=' ',index=False,header=False)

In [ ]:
#uncomment to run
# convert_yolo_to_obb(yolo_dataset_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels",
#                     output_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels")

In [ ]:
# uncomment to run
# convert_obb_to_yolo(obb_dataset_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels",
#                     output_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels")